In [1]:
import os
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np
from pydub import AudioSegment
from torch.utils.data import Dataset, DataLoader
from faster_whisper import WhisperModel


c:\ProgramData\anaconda3\envs\DL\Lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# === Настройки ===
MODEL_NAME = "Systran/faster-whisper-small"
LANGUAGE = "ru"
AUDIO_DIR = "train_opus/audio"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32

In [3]:
# # === Класс датасета ===
# class OpusDataset(Dataset):
#     def __init__(self, audio_dir):
#         self.files = [
#             os.path.join(audio_dir, f)
#             for f in os.listdir(audio_dir)
#             if f.endswith(".opus") and not f.startswith("._")
#         ]
#         self.sampling_rate = 16000

#     def __len__(self):
#         return len(self.files)

#     def __getitem__(self, idx):
#         path = self.files[idx]
#         audio = AudioSegment.from_file(path)
#         audio = audio.set_frame_rate(self.sampling_rate).set_channels(1)
#         samples = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0
#         return samples, os.path.basename(path)

In [4]:
# # === Collate-функция ===
# def collate_fn(batch):
#     samples, filenames = zip(*batch)
#     return samples, filenames

In [5]:
# dataset = OpusDataset(AUDIO_DIR)
# loader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, num_workers=0)

In [6]:
# === Загружаем модель ===
print("📥 Загружаю модель...")
# compute_type="float16" — быстрее на GPU
model = WhisperModel(
    MODEL_NAME,
    device=DEVICE,
    compute_type="float16" if DEVICE == "cuda" else "int8"
)

📥 Загружаю модель...


In [18]:
files = [
    os.path.join(AUDIO_DIR, f)
    for f in os.listdir(AUDIO_DIR)
    if f.endswith(".opus") and not f.startswith("._")
]
files.sort()

In [ ]:
files = files[:1000]


data = []
for id in tqdm(files, desc="Распознавание (по одному файлу)"):
    try:
        # faster-whisper сам декодирует и ресемплит, удобно подавать путь к файлу
        segments, _ = model.transcribe(
            id,
            language=LANGUAGE,
            beam_size=5,
            vad_filter=True  # чуть устойчивее к тишине/шумам
        )
        text = " ".join(seg.text.strip() for seg in segments).strip()
        data.append({"filename": os.path.basename(id), "text": text})
    except Exception as e:
        data.append({"filename": os.path.basename(id), "text": f"[ERROR] {e}"})


Распознавание (по одному файлу): 100%|██████████| 1000/1000 [06:47<00:00,  2.46it/s]


In [8]:
# Сохраняем результат
df = pd.DataFrame(data)
df.to_csv("train_transcriptions.csv", index=False, encoding="utf-8")

In [9]:
df

,filename,text
0,0000020974966386734278978724803273402299.opus,Что-то я не слышу энтузиазма в голосе.
1,0000080688513237533207488522718221342719.opus,"Так, колонки подключу. Да подожди, я не слышу ..."
2,0000168205906481357162070811351230430612.opus,"Да, Алочка, да, очень свежтой."
3,0000283353856125472766020523736784290228.opus,"Нормально все было, я просто хотела послушать,..."
4,0000794176926750993969039908245303376577.opus,"Чуть глубже трёх меня разрывают, а продусы нег..."
...,...,...
995,0105238937291517403149073557607199530162.opus,
996,0105239455698711277747798516509814424361.opus,не слышно
997,0105249533398052829276540100185502771712.opus,"Ничего не слышно по поводу, но у нас будет сей..."
998,0105301607707881572976156310017898408991.opus,"На земле слышно, а в самолете не слышно."


In [34]:
mismatched_id = '0012068190892455824387393277244528645018'
df[df['filename']==f'{mismatched_id}.opus'].text

113    Если жёст! Так, альбисии! И на его все!
Name: text, dtype: object

In [ ]:
submission_df = pd.DataFrame()
submission_df['id'] = df['filename'].str.replace(r'\.opus$', '', regex=True)
submission_df['label'] = df['text'].str.contains(r'не\s*слыш(у|но)', case=False, regex=True).astype(int)

C:\Users\Степан\AppData\Local\Temp\ipykernel_18376\1566910217.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['text'].str.contains(r'не\s*слыш(у|но)', case=False, regex=True, na=False) |


In [11]:
import json

# читаем JSON-файл
with open("train_opus/word_bounds.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# проверим, что это словарь
if isinstance(data, dict):
    keys = set(data.keys())
    print(keys)
else:
    print("JSON не является словарем")

{'...'}


In [ ]:
# train = pd.DataFrame({
#     "id": [os.path.basename(f).replace(".opus", "") for f in files],
# })

# train["label"] = train["id"].apply(lambda x: 1 if x in keys else 0)

In [41]:
submission_df["y_train"] = submission_df["id"].apply(lambda x: 1 if x in keys else 0)

In [42]:
# количество несовпадений
mismatches = submission_df[submission_df["label"] != submission_df["y_train"]]

# вывести количество и список id
count_mismatches = len(mismatches)
mismatch_ids = mismatches["id"].tolist()

print("Количество несовпадений:", count_mismatches)
print("Список несовпавших id:", mismatch_ids)


Количество несовпадений: 105
Список несовпавших id: ['0000920928762365426001926574248557079166', '0003499114203020773015064821233235825382', '0003599419971760967218888800546393281812', '0007314369356834137746083411926655898377', '0009208269603329217951953363163156553594', '0010674686573632000877624012331907490053', '0011730198411433612237692315235258469498', '0012068190892455824387393277244528645018', '0012453303476085815882884482522394607241', '0012945761482967913622335814672158012103', '0013828848543422647687279016527837421176', '0016132772855380378865795753494213108351', '0016319110396791545381348118805491710431', '0017442403288585671330156997870862765694', '0020369882654891096832418400537496290073', '0020715778571188393181283384133105106152', '0022376907125976586515019494156895889497', '0022609473026451468196406689245184357752', '0023742535085614417999755913378850455409', '0025083889151292074849458059957404885610', '0026224129806758861794736831585723786303', '0028491571469058300681

In [43]:
from collections import Counter

tp = ((submission_df["label"] == 1) & (submission_df["y_train"] == 1)).sum()
tn = ((submission_df["label"] == 0) & (submission_df["y_train"] == 0)).sum()
fp = ((submission_df["label"] == 1) & (submission_df["y_train"] == 0)).sum()
fn = ((submission_df["label"] == 0) & (submission_df["y_train"] == 1)).sum()

conf_map = {"TP": tp, "FP": fp, "FN": fn, "TN": tn}
print(conf_map)


{'TP': 450, 'FP': 53, 'FN': 52, 'TN': 445}


In [ ]:
import subprocess
from pathlib import Path

def convert_opus_to_mp3(file_paths: list[str]):
    output_dir = Path("mismatched_mp3")
    output_dir.mkdir(exist_ok=True)

    for id in file_paths:
        input_file = Path(f'train_opus/audio/{id}.opus')
        if not input_file.exists():
            print(f"⚠️ Файл не найден: {input_file}")
            continue

        output_file = output_dir / (input_file.stem + ".mp3")

        cmd = [
            "ffmpeg",
            "-y",  # перезаписывать при необходимости
            "-i", str(input_file),
            "-vn",  # без видео
            "-c:a", "libmp3lame",
            "-q:a", "2",  # качество (0 — лучше, 9 — хуже)
            str(output_file)
        ]

        subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"✅ Конвертировано: {output_file}")

# Пример использования:
# convert_opus_to_mp3(["audio1.opus", "audio2.opus", "path/to/audio3.opus"])
convert_opus_to_mp3(mismatch_ids)

✅ Конвертировано: mismatched_mp3\0000920928762365426001926574248557079166.mp3
✅ Конвертировано: mismatched_mp3\0003499114203020773015064821233235825382.mp3
✅ Конвертировано: mismatched_mp3\0005356541005052807824434036870626881651.mp3
✅ Конвертировано: mismatched_mp3\0005823401365407084383267460437392633853.mp3
✅ Конвертировано: mismatched_mp3\0009208269603329217951953363163156553594.mp3
✅ Конвертировано: mismatched_mp3\0011730198411433612237692315235258469498.mp3
✅ Конвертировано: mismatched_mp3\0012068190892455824387393277244528645018.mp3
✅ Конвертировано: mismatched_mp3\0012453303476085815882884482522394607241.mp3
✅ Конвертировано: mismatched_mp3\0012945761482967913622335814672158012103.mp3
✅ Конвертировано: mismatched_mp3\0017192528527784640188431271487669791237.mp3
✅ Конвертировано: mismatched_mp3\0017442403288585671330156997870862765694.mp3
✅ Конвертировано: mismatched_mp3\0020300756426902777519012035303603641693.mp3
✅ Конвертировано: mismatched_mp3\0020715778571188393181283384133

In [12]:
submission_df.to_csv('submission1.csv', index=False)